In [ ]:
!pip install -q \
  llama-index \
  llama-index-embeddings-huggingface \
  llama-index-llms-huggingface \
  transformers accelerate sentence-transformers pypdf bitsandbytes


In [18]:
!pip install -q gradio

In [2]:
from google.colab import files

uploaded_files = files.upload()


Saving Indian constitution.pdf to Indian constitution.pdf


In [3]:
from pypdf import PdfReader
from llama_index.core import Document

documents = []

for filename in uploaded_files:
    reader = PdfReader(filename)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""

    if text.strip():
        documents.append(Document(text=text))

print(f"Loaded {len(documents)} document(s)")


Loaded 1 document(s)


In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-base-en-v1.5"
)


In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
import torch

Settings.llm = HuggingFaceLLM(
    model_name="Qwen/Qwen2-7B-Instruct",
    tokenizer_name="Qwen/Qwen2-7B-Instruct",
    device_map="auto",
    model_kwargs={
        "load_in_4bit": True,
        "bnb_4bit_compute_dtype": torch.float16,
        "bnb_4bit_use_double_quant": True,
        "bnb_4bit_quant_type": "nf4",
    },
    generate_kwargs={
        "temperature": 0.1
    },
    is_chat_model=True,
)


In [6]:
Settings.chunk_size = 512
Settings.chunk_overlap = 50


In [7]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

print("Vector index built successfully")


Vector index built successfully


In [8]:
from llama_index.core.prompts import PromptTemplate

LEGAL_PROMPT = PromptTemplate(
"""
Answer ONLY using the provided context.
If the answer is not present, reply exactly:
"The uploaded document does not contain information relevant to this query."

Context:
{context_str}

Question:
{query_str}

Answer:
"""
)


In [21]:
from llama_index.core.response_synthesizers import ResponseMode

query_engine = index.as_query_engine(
    similarity_top_k=2,
    response_mode=ResponseMode.COMPACT,
    text_qa_template=LEGAL_PROMPT,
)


In [19]:
def chat_with_lexibot(user_query):
    if not user_query.strip():
        return "Please enter a valid question."

    answer = str(query_engine.query(user_query)).strip()

    if len(answer) < 40:
        return "The uploaded document does not contain information relevant to this query."

    return answer



In [20]:
import gradio as gr

with gr.Blocks(title="LexiBot ⚖️") as demo:
    gr.Markdown(
        """
        # ⚖️ LexiBot
        **Legal Document Assistant**
        Upload documents above, then ask legal questions below.
        """
    )

    chatbot = gr.Chatbot(height=350)

    user_input = gr.Textbox(
        placeholder="Ask a legal question...",
        show_label=False
    )

    def respond(message, history):
        response = chat_with_lexibot(message)
        history.append((message, response))
        return history, ""

    user_input.submit(respond, [user_input, chatbot], [chatbot, user_input])

demo.launch()


/tmp/ipython-input-3627199367.py:12: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=350)
/tmp/ipython-input-3627199367.py:12: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=350)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e5d6444b39c36ed49f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
